In [1]:
%load_ext watermark
%watermark

Last updated: 2022-02-24T14:49:42.681564+01:00

Python implementation: CPython
Python version       : 3.7.6
IPython version      : 7.12.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 23 Model 8 Stepping 2, AuthenticAMD
CPU cores   : 12
Architecture: 64bit



In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [3]:
from scipy.optimize import curve_fit


import tkinter as tk
from tkinter import filedialog
%matplotlib qt

In [4]:
%run fct_RF_Heating
matplotlib.rcParams.update({'font.size': 25})

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
z0 = r0/np.sqrt(2)     # longueur piège Ca+
d0 = np.sqrt(0.5*r0**2 + z0**2)
print(r0,z0,d0)

Omega = 2.05e6*2*pi # 2.047e6

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

0.0025 0.0017677669529663686 0.0025
$\kappa_{simion}$ = 0.422611


In [99]:
# beta non adiabatique
# for a given q_x, compute Urf, beta_x, omega_x
# and Udc for spherical cloud alpha=1
q = linspace(0,1,11)
a = []
print('|   q  |    Urf   ||   a   |   Udc   |')
for i,j in enumerate(q):
    a.append(0.5*j**2) # 0.5*j**2
    Urf = -j*m_Ca*d0**2*Omega**2/(2*C_e)
    Udc = 0.5*a[-1]*Urf/j
#     Udc = a[-1]*m_Ca*d0**2*Omega**2/(8*C_e)
    print(f'| {j:.02f} | {Urf:08.03f} || {a[-1]:.03f} | {Udc:07.03f} |')

|   q  |    Urf   ||   a   |   Udc   |
| 0.00 | -000.000 || 0.000 | 0000nan |
| 0.10 | -021.538 || 0.005 | -00.538 |
| 0.20 | -043.076 || 0.020 | -02.154 |
| 0.30 | -064.615 || 0.045 | -04.846 |
| 0.40 | -086.153 || 0.080 | -08.615 |
| 0.50 | -107.691 || 0.125 | -13.461 |
| 0.60 | -129.229 || 0.180 | -19.384 |
| 0.70 | -150.768 || 0.245 | -26.384 |
| 0.80 | -172.306 || 0.320 | -34.461 |
| 0.90 | -193.844 || 0.405 | -43.615 |
| 1.00 | -215.382 || 0.500 | -53.846 |


H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [100]:
# secular frequencies
beta_guess = 0.5

for l in range(10):
    beta_guess = beta_continue_alamano(a[3],q[3],beta_guess)
    beta_guess = sqrt(beta_guess)
#     print(beta_guess,beta_nonadia)
print(f'beta_r = {beta_guess:.3f}, f_r = {Omega/2*beta_guess/2/pi:.3f} Hz')
beta_guess = 0.5
for l in range(10):
    beta_guess = beta_continue_alamano(2*a[3],2*q[3],beta_guess)
    beta_guess = sqrt(beta_guess)
print(f'beta_z = {beta_guess:.3f}, f_z = {Omega/2*beta_guess/2/pi:.3f} Hz')

beta_r = 0.307, f_r = 314322.213 Hz
beta_z = 0.578, f_z = 592829.640 Hz


# Extract data as npz

In [7]:
file_path = load_file_GUI('/home/adrian/RemoteFS/Rivendel/Simulations/20220221/ST03_RF03/Try00')
print(file_path)

('K:/Universite_Simus/20220221/ST03_RF03_Gam00/Temp_SimuType0_N01024_Vrf0064_Vst-0.5948D+01V_nt1000.dat',)


In [291]:
address = '/home/adrian/RemoteFS/Rivendel/Simulations/20220221/ST03_RF03/Try00'
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
print(sort(onlyfiles)[0].strip('.dat'))

Temp_SimuType0_N00064_Vrf0064_Vst-0.5948D+01V_nt1000


In [106]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

## Data loading : Time and Temperature
data_name, num_runs, Time, Temperature, Size_w, Size2 \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

> Répertoire : K:/Universite_Simus/20220221/ST03_RF03_Gam00
> Filename : Size_SimuType0_N01024_Vrf0064_Vst-0.5948D+01V_nt1000.dat
> myslashpos | [2, 19, 28]
> slashcond | -1
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 4
['']
> condition names ['']
> number of points 1
> N_ions = 1024
> e_GMol = 50
> Points | 1
> Simulations pour chaque point | ['Universite_Simus/20220221\\ST03_RF03_Gam00', 'Universite_Simus/20220221\\ST03_RF03_Gam01', 'Universite_Simus/20220221\\ST03_RF03_Gam02', 'Universite_Simus/20220221\\ST03_RF03_Gam03']
Hello
K:/Universite_Simus/20220221\ST03_RF03_Gam00
['Size_SimuType0_N01024_Vrf0064_Vst-0.5948D+01V_nt1000.dat', 'Temp_SimuType0_N01024_Vrf0064_Vst-0.5948D+01V_nt1000.dat', 'trj_L.dat']
['trj_L.dat']
Point n° 0
00 - 00  >  K:/Universite_Simus/20220221\ST03_RF03_Gam00/Size_SimuType0_N01024_Vrf0064_Vst-0.5948D+01V_nt1000
K:/Universite_Simus/20220221\ST03_RF03_Gam01
['Size_SimuType0_N01024_Vrf0064_Vst-0.5948D+01V_nt1000.d

In [107]:
all_subdir   = data_name[0][0] 
all_size_dir = data_name[1][0]
all_temp_dir = data_name[2][0]
all_xva_dir  = data_name[3][0]
all_trj_dir  = data_name[3][0]

i_fly_Verlet_init = 1e2
i_fly_Verlet      = 1e4

In [111]:
test_condition = 1
rep_name     = re.sub('\\\\','/',all_subdir[test_condition])
str_load_Ver_xva = re.sub('\\\\','/',all_xva_dir[test_condition])
str_load_Ver_T   = re.sub('\\\\','/',all_temp_dir[test_condition])
str_load_Ver_size   = re.sub('\\\\','/',all_size_dir[test_condition])
N_ions = 1024
print(rep_name)

K:/Universite_Simus/20220221/ST03_RF03_Gam01


In [112]:
# plot end Verlet
iRF, save_xva = load_xva_init_bin_Ver(str_load_Ver_xva, N_ions)
fig = plt.figure('Fin Verlet',clear='True')
fig.set_size_inches(10, 5)
ax = subplot(121)
ax.scatter(save_xva[0,:]*1e6,save_xva[1,:]*1e6,marker='o')
ax.set_xlabel('x [µm]')
ax.set_ylabel('y [µm]')
ax.grid()
# ax.set_xlim(-2e8,2e8)
# ax.set_aspect(1)
ax = subplot(122)
ax.scatter(save_xva[2,:]*1e6,save_xva[1,:]*1e6,marker='o')
ax.set_xlabel('z [µm]')
# ax.set_ylabel('y [µm]')
ax.grid()
# ax.set_aspect(1)
tight_layout()

In [113]:
# plot temperature
data = loadtxt('{}.dat'.format(str_load_Ver_T),comments='%');
time = data[:,0]
T_CM = data[:,1:4]
T_aux = data[:,4:7] # tt, T_CM, T_aux, PM

fig = plt.figure('Temperature',clear='True')
fig.set_size_inches(10, 5)
ax = subplot(111)
ax.plot(time*1e3,mean(T_aux, axis=1),color='r')
ax.set_xlabel('time [ms]')
ax.set_ylabel('T [K]')
ax.grid()
tight_layout()

In [366]:
fig = plt.figure('FFT Temperature',clear='True')
fig.set_size_inches(10, 5)
ax = subplot(111)
sp = np.fft.fft(mean(T_aux, axis=1))
freq = np.fft.fftfreq(time.shape[-1], d=2*pi/(Omega))
ax.scatter(freq*1e-3, sp.real,marker='+')
# ax.set_xlim(1e8,5e8)
ax.set_ylim(-1e1,1e4)
ax.set_xlabel('freq [kHz]')
# ax.set_ylabel('T [K]')
ax.grid()
tight_layout()

In [115]:
# plot size s Blümel
data = loadtxt('{}.dat'.format(str_load_Ver_size),comments='%');
sx2 = data[:,0]
sy2 = data[:,1]
sz2 = data[:,2]
s2 = data[:,3]

fig = plt.figure('size',clear='True')
fig.set_size_inches(10, 5)
ax = subplot(111)
ax.plot(time*1e3,s2,color='r')
ax.set_xlabel('time [ms]')
ax.set_ylabel('s^2 [?]')
ax.grid()
tight_layout()

In [118]:
diff(time)

array([5.e-07, 5.e-07, 5.e-07, ..., 5.e-07, 5.e-07, 5.e-07])

In [171]:
for j,k in enumerate(data_name[0]):
    cond = k[0][52:61]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_'+str(cond)
    np.savez( outfile, time=Time[j], temp = mean(Temperature[j],axis=0)) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)

ST03_RF03


In [172]:
# load check
cond = 'ST03_RF03'
outfile = 'Time_and_temp_RFHEAT_'+str(cond)
with load(outfile+'.npz') as data:
    a = data['time']
    b = data['temp']
print(shape(a))
print(shape(b))

(100000,)
(100000,)


In [81]:
whos

Variable                                       Type                          Data/Info
--------------------------------------------------------------------------------------
C_e                                            float                         1.602e-19
Omega                                          float                         12880529.879718151
Path                                           type                          <class 'pathlib.Path'>
T_max                                          int                           50
T_thr                                          int                           1
Temperature                                    list                          n=24
Time                                           list                          n=24
Urf                                            list                          n=25
a                                              ndarray                       100000: 100000 elems, type `float64`, 800000 bytes (781.25 kb

# Work with data

In [175]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python')
print(file_path)

('/home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_ST03_RF03.npz',)


In [177]:
# load check
# cond = 'DC11_RF00'
# outfile = 'Time_and_temp_RFHEAT_'+str(cond)
Time = []
Temperature = []
for j,k in enumerate(file_path):
    with load(k) as data:
        Time.append( data['time'] )
        Temperature.append( data['temp'] )
        print(data['time'])
        print(data['temp'])
    print(j,k)
print(shape(Time))
print(shape(Temperature))

[0.00050062 0.00050112 0.00050162 ... 0.05049912 0.05049962 0.05050012]
[0.00180059 0.00155639 0.00201779 ... 0.00173517 0.00178356 0.00177177]
0 /home/adrian/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_ST03_RF03.npz
(1, 100000)
(1, 100000)


In [178]:
Urf = [64.62]
Vst = [5.948]

In [454]:
imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(10); clf();
i = 0
print(i,Urf[i])
#     subplot(1,9,i+1)
semilogy(Time[i]*1e3,Temperature[i],label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
grid()
legend(fontsize=12)

imax 1


IndexError: invalid index to scalar variable.

In [455]:
imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(1); clf();
for i in range(0,imax-1):
    print(i)
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))
    k_min = k_thr - (k_max - k_thr)
    if k_min < 0:
        k_min = 0

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

#     subplot(1,9,i+1)
    subplot(imax//3,4,i+1)
    semilogy(Time[i][k_min:k_max]*1e3,Temperature[i][k_min:k_max],label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
    grid()
    legend(fontsize=12)

TypeError: object of type 'numpy.float64' has no len()

In [97]:
def func2(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+(t-t0)**2)**D + B

def dfunc2_dt(t, t0, A, B,  C, D):
    return 10**(A*(C + (t - t0)**2)**(-D)*(t - t0) + B)*(-A*D*(C + (t - t0)**2)**(-D)*(t - t0)*(2*t - 2*t0)/(C + (t - t0)**2) + A*(C + (t - t0)**2)**(-D))*log(10)

In [704]:
T_max = 200
T_thr = 1
fit_param = zeros((imax,5))
fit_pcov  = zeros((imax,5,5))
my_order = []

figure(1); clf()
for i in range(0,imax):
    print('<<<',i,'>>>')
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

    k_min = k_thr - int(2*(k_max - k_thr))
    if k_min < 0: k_min = 0
#         print(k_min)

    x  = Time[i][k_min:k_max]
    y  = Temperature[i][k_min:k_max]
    x0 = Time[i][k_thr]

    A0  = 1
    B0  = log10(Temperature[i][k_thr])
    C0  = 1.0e-8
    D0  = 0.5
    p0 = array([x0, A0, B0,  C0, D0])

    try:
        popt, pcov = curve_fit(func2, x, log10(y), p0, maxfev= 500000)
        fit_param[i,:] = popt[0], popt[1], popt[2], popt[3], popt[4]
        fit_pcov[i,:]  = pcov

        y_fit = func2(x,popt[0], popt[1], popt[2], popt[3], popt[4])
        y_fit = 10**y_fit

        subplot(imax//3,4,i+1)
        semilogy(x*1e3, y    ,label=' Vrf = '+str(Urf[i]))
        semilogy(x*1e3, y_fit,label='Fit')
        grid()
        legend(fontsize=10)
        # select cases
        # non zero fit value
        my_order.append(i)
    except TypeError:
        print('TypeError for ',Urf[i])

<<< 0 >>>
<<< 1 >>>
<<< 2 >>>
<<< 3 >>>
<<< 4 >>>
<<< 5 >>>


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in power
  


<<< 6 >>>
<<< 7 >>>
<<< 8 >>>
<<< 9 >>>
<<< 10 >>>
<<< 11 >>>
<<< 12 >>>
<<< 13 >>>
<<< 14 >>>
<<< 15 >>>
<<< 16 >>>
<<< 17 >>>
TypeError for  40.0
<<< 18 >>>
TypeError for  45.0
<<< 19 >>>
TypeError for  50.0
<<< 20 >>>
TypeError for  55.0
<<< 21 >>>
<<< 22 >>>
<<< 23 >>>
<<< 24 >>>
TypeError for  85.0


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [705]:
# select only Urf values
# for which non zero fit
Urf_order = [Urf[x] for w,x in enumerate(my_order)]
# sort my_order
# so Urf is ascending
zipped_lists = zip(Urf_order, my_order)
sorted_zipped_lists = sorted(zipped_lists)
for i,j in enumerate(sorted_zipped_lists):
    my_order[i] = j[1]
# delete manually
# other innapropriate values
my_order.remove(23)
Urf_order.remove(80.0)
my_order.remove(22)
Urf_order.remove(75.0)
Urf_order = list(sort([Urf[x] for w,x in enumerate(my_order)]))
print(Urf_order)
print(my_order)

# resort all variables
# in order so
# Urf ascending
imax_order = len(Urf_order)
fit_param_order = zeros((imax_order,5))
fit_pcov_order  = zeros((imax_order,5,5))
Time_order = []
Temperature_order = []
for i,j in enumerate(my_order):
    fit_param_order[i,:] = fit_param[j,:]
    fit_pcov_order[i,:]  = fit_pcov[j,:]
    Time_order.append(Time[j])
    Temperature_order.append(Temperature[j])

[60.0, 61.0, 62.0, 62.2, 62.4, 62.6, 62.8, 63.0, 63.2, 63.4, 63.6, 63.8, 64.0, 65.0, 66.0, 67.0, 68.0, 70.0]
[0, 1, 2, 9, 10, 11, 12, 3, 13, 14, 15, 16, 4, 5, 6, 7, 8, 21]


In [725]:
# draw one temp and fit
figure(3); clf()
cm = pylab.get_cmap('jet')

i = 9
print(Urf_order[i])
# print(fit_pcov_order[i,:])
y_fit = func2(Time_order[i],
              fit_param_order[i,0],
              fit_param_order[i,1],
              fit_param_order[i,2],
              fit_param_order[i,3],
              fit_param_order[i,4])
y_fit = 10**y_fit
subplot(121)
plot((Time_order[i])*1e3,Temperature_order[i],color='r')
plot((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
ylabel('T [K]')
grid()

# xlim(3.6,3.7)
# ylim(-1,10)
subplot(122)
semilogy((Time_order[i])*1e3,Temperature_order[i],color='r')
semilogy((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
grid()
tight_layout()

63.4


In [706]:
str_labels = ['t0', 'A', 'B', 'C', 'D']
figure('Param'); clf();
for i in range(0,5):
    subplot(1,5,i+1)
    if i == 2:
        plot(Urf_order,10**(fit_param_order[:,i]),'o-', label=str_labels[i])
    else:
        plot(Urf_order,(fit_param_order[:,i]),'o-', label=str_labels[i])
    xlabel('Urf')
    grid()
    legend()

In [707]:
figure(29); clf()
cm = pylab.get_cmap('jet')

i = 0
for i in range(0,imax_order):
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
    semilogy((Time_order[i]-t0)*1e3,dy/y,color=cm(i/imax))
    
i = 1
for i in range(0,imax_order):
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
    semilogy((Time_order[i]-t0)*1e3,dy/y,'--',color=cm(i/imax))

In [716]:
xa = -0.6
xb = 0.6
ya = 6e-6
yb = 1e0
figure(30); clf()
ax = subplot(111)
for i in range(0,imax_order):
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
    ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.set_xlabel('$t-t_0$ [ms]')
    ax.set_ylabel('Heating rate [K/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
    ax.set_xlim([xa,xb])
    ax.set_ylim([ya,yb])
    grid()
    
locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
ax.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
ax.yaxis.set_minor_locator(locmin)
ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [298]:
xa = -0.05
xb = 0.05
ya = 1e-2
yb = 6e-1
figure(31); clf()
ax = subplot(111)
for i in range(0,imax_order):
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
    ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6/y,label=str(Urf[i]),color=cm(i/imax_order))
    ax.set_xlabel('$t-t_0$ [ms]')
    ax.set_ylabel('Heating rate / T\n[1/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
    ax.set_xlim([xa,xb])
    ax.set_ylim([ya,yb])
    grid()
    
locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
ax.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9),numticks=12)
ax.yaxis.set_minor_locator(locmin)
ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.02))
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

# t0 and T as a function of Urf

In [728]:
t0    = zeros(imax_order)
y_t0  = zeros(imax_order)
dy_t0 = zeros(imax_order)
for i in range(0, imax_order):
    t0[i], A, B, C, D = fit_param_order[i,:]
    y  = func2(t0[i],t0[i], A, B, C, D);
    y_t0[i]  = 10**y
    dy_t0[i] = dfunc2_dt(t0[i], t0[i], A, B,  C, D)

In [729]:
def my_lin(x,a,b):
    return a*x+b
def my_power_law(x,a,b):
    return a*(x**-b)

In [730]:
# linear plot
figure(321); clf();
subplot(111)
plot(Urf_order,t0*1e3,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

In [731]:
# log plot
figure(321); clf();
subplot(111)
loglog(Urf_order,t0*1e3,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

In [763]:
# find the coefficient of power law
xx = Urf_order
yy = (t0)*1e3 # (yy-min(t0))*1e3
# xx = xx[1:]
# yy = -yy[1:]
popt, pcov = curve_fit(my_lin,log(xx),log(yy))

k_fromlog = popt[0]
B_fromlog = popt[1]
print(k_fromlog,B_fromlog)
# plot data
# linearized with corresponding power -popt[0]

figure(3210); clf();
ax = subplot(111)
ax.plot(xx,yy,ls='',marker='+',mew=3,ms=10)
ax.grid()
# ax.set_xlabel(f'$U_{{rf}}^{{{popt[0]:.2e}}}$ [V]')
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'
# ax.errorbar(popt[0],yy,yerr = 1e-2,ls='') # sqrt(fit_pcov_order[:,0,0])*1e7

# fit the power law
popt, pcov = curve_fit(my_power_law,xx,yy,p0=[1e23,12.5],maxfev=2500)
print(popt)
xx_fit = linspace(59.8,71,100)
ax.plot(xx_fit,my_power_law(array(xx_fit),popt[0],popt[1]),
    color='r',ls='--',label='Direct pow. Law fit')
ax.plot(xx_fit,my_power_law(array(xx_fit),B_fromlog,k_fromlog),
    color='b',ls=':',label='From log')
# ax.plot(linspace(0.5,72,1000),my_power_law(linspace(0.5,72,1000),1e23,12.5),
#     color='g',ls='--',label='test pow. Law')

ax.legend()
# ax.text(4.15,2,f'$a={popt[0]:.2f}, b={popt[1]:.2f}$',
#             color='xkcd:black',fontsize='20',weight="bold",
#             ha='left',va='center')

# ax.set_xlim(-2,72)
# ax.set_ylim(-1,18)
tight_layout()

-15.233803215236648 64.74541613417573
[5.23738105e+44 2.44715990e+01]


In [661]:
10**B_fromlog

5.564371713038298e+64

In [ ]:
# find the coefficient of power law
xx = Urf_order - Urf_order[0]
yy = (t0)*1e3 # (yy-min(t0))*1e3
xx = xx[1:]
yy = -yy[1:]

popt, pcov = curve_fit(my_lin,log(xx),log(yy))
print(popt[0],10**(popt[1]))

# plot data
# linearized with corresponding power -popt[0]

figure(3210); clf();
ax = subplot(111)
ax.plot(xx**popt[0],yy,ls='',marker='+',mew=3,ms=10)
ax.grid()
ax.set_xlabel(f'$U_{{rf}}^{{{popt[0]:.2e}}}$ [V]')
ax.set_ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'
print(fit_pcov_order[5,:])
ax.errorbar(xx**popt[0],yy,yerr = 1e-2,ls='') # sqrt(fit_pcov_order[:,0,0])*1e7

# fit the power law
popt, pcov = curve_fit(my_lin,xx**popt[0],yy,p0=[1e25,0])
xx_fit = linspace(0.7e-28,8.3e-28,100)
ax.plot(xx_fit,my_lin(array(xx_fit),popt[0],popt[1]),
    color='k',ls='--')


# ax.text(4.15,2,f'$a={popt[0]:.2f}, b={popt[1]:.2f}$',
#             color='xkcd:black',fontsize='20',weight="bold",
#             ha='left',va='center')

tight_layout()

In [526]:
yy_moy = mean(yy)
residual = 0
tot = 0
for i,j in enumerate(yy):
    index_for_res = Urf[i]**
    residual += yy - my_lin(array(xx_fit)[10],popt[0],popt[1])
    tot += yy - yy_moy
    R_param = residual/tot

1.5680162450037478e-07

In [304]:
# 1/sqrt() plot
figure(321); clf();
ax = subplot(111)
ax.plot(Urf_order,(t0*1e3)**-0.5,ls='-',marker='+',mew=5,ms=10)
ax.grid()
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$1/\sqrt{t_0}$ [ms]') # '$T(t_0) / t_0$ [K/s]'

popt, pcov = curve_fit(my_lin,Urf_order,(t0*1e3)**-0.5)
ax.plot(Urf_order,my_lin(array(Urf_order),popt[0],popt[1]),
    color='k',ls='--')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

ax.text(66.3,0.75,f'$1/\sqrt{{t_0}} = {popt[0]:.2e} U_{{rf}} + {popt[1]:.2f}$',
            color='xkcd:black',fontsize='20',weight="bold",
            ha='right',va='center')

tight_layout()

In [305]:
figure(322); clf();
ax = subplot(111)
ax.plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
ax.grid()
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

popt, pcov = curve_fit(my_lin,Urf_order,y_t0)
ax.plot(Urf_order,my_lin(array(Urf_order),popt[0],popt[1]),
    color='k',ls='--')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

ax.text(64,0.815,f'$T(t_0) = {popt[0]:.2e} U_{{rf}} + {popt[1]:.2f}$',
            color='xkcd:black',fontsize='20',weight="bold",
            ha='left',va='center')

tight_layout()

# identique à 10**B

In [306]:
figure(323); clf();
subplot(111)
plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

# identique à 10**B

In [ ]:
savefig('T_t0_Udc.png')

In [79]:
figure(33); clf();
subplot(121)
title('Vrf = '+str(Urf[2]))
plot(Udc,t0_v,'o')
grid()
xlabel('Udc[V]')
ylabel('T[t = t0] / t0 [K/s]')

subplot(122)
title('Vrf = '+str(Urf[3]))
plot(Udc,t1_v,'o')
grid()
xlabel('Udc[V]')
ylabel('T[t = t0] / t0 [K/s]')


NameError: name 'Udc' is not defined

In [223]:
figure(33); clf();
subplot(121)
title('Vrf = '+str(Urf[2]))
plot(Udc,y_t0,'o')
grid()
xlabel('Udc[V]')
ylabel('T[t = t0]')

subplot(122)
title('Vrf = '+str(Urf[3]))
plot(Udc,y_t1,'o')
grid()
xlabel('Udc[V]')
ylabel('T[t = t0]')


Text(0, 0.5, 'T[t = t0] / t0 [K/s]')

In [80]:
figure(31); clf()

i = 0
for j in range(0,jmax):
    t0, A, B, C, D = fit_param[i,j,:]
    y  = func2(time,t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(time, t0, A, B,  C, D)
    semilogy(time-t0,y)
    
i = 1
for j in range(0,jmax):
    t0, A, B, C, D = fit_param[i,j,:]
    y  = func2(time,t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(time, t0, A, B,  C, D)
    title('Vrf = '+str(Urf[i+2]))
    semilogy(time-t0,y,'--')

NameError: name 'jmax' is not defined